# Find elements in folder of StructureDefinitions

- for hard coded path search all SD for mandatory (min=1) elements
- list em in a md table format in std out

In [65]:
from pathlib import Path
from json import load, dumps
my_path = Path('/Users/ehaas/Documents/FHIR/US-Core/output')
my_path
us_core_gran_controls = ['health-concern','encounter-diagnosis','problem-list-item','sdoh','social-history','laboratory','survey','vital-signs',]
white_list = ['AllergyIntolerance',
 'CarePlan',
 'CareTeam',
 'Coverage',
 'Device',
 'DiagnosticReport',
 'DocumentReference',
 'Encounter',
 'Goal',
 'Immunization',
 'Location',
 'Medication',
 'MedicationDispense',
 'MedicationRequest',
 'Organization',
 'Patient',
 'Practitioner',
 'PractitionerRole',
 'Procedure',
 'Provenance',
 'QuestionnaireResponse',
 'RelatedPerson',
 'ServiceRequest',
 'Specimen']
# white_list = ["Observation"]
expansions = my_path / "expansions.json"
expansions_obj = load(expansions.open())

### Get Codes from ValueSet
  - using the canonical uri fetch the codes from the ig builds expaansions.json file: 
      `/Users/ehaas/Documents/FHIR/US-Core/output/expansions.json`

   - path =  Bundle.entry.where(fullUrl = canoninical).resource.contains[].code (or system)

In [ ]:
def get_expansion(canonical):
  for entry in expansions_obj['entry']:
      if entry['fullUrl'] == canonical:
          # print(entry['fullUrl'])
          return [f"{contains['code']}({contains['display']})" for contains in entry['resource']['expansion']['contains']]


In [68]:
my_profiles = set()
profile_with_category_binding = set()
table_rows = []
print('|Profile|Category Code or Value Set|')
print('|---|---|')
for i in my_path.glob('Struct*.json'):
    # print(i)
    obj = load(i.open())
    # my_profiles.add(obj['title'])
    profile_type = obj['snapshot']['element'][0]['id']
    # print(profile_type)
    # print(f"=== {obj['title']} ===")
    # find category codes in 
    if profile_type in white_list:
      my_profiles.add(obj['title'])
      for diff in obj['differential']['element']:
          # print (diff['path'])
          try:
              if 'category' in  diff['path']:
                  # print(i)
                  # print('=== Category! ===')
                  my_code = diff['patternCodeableConcept']['coding'][0]['code']
                  if my_code in us_core_gran_controls:
                    my_code = f"**{my_code}**" 
                  table_rows.append(f"{obj['title']}|{my_code}|")
                  profile_with_category_binding.add(obj['title'])

          except KeyError as e:
              try:
                my_code = diff['fixedCode']
                if my_code in us_core_gran_controls:
                    my_code = f"**{my_code}**" 
                    table_rows.append(f"{obj['title']}|{my_code}|")
                    profile_with_category_binding.add(obj['title'])
              except KeyError as e:
                try:
                  # print(diff['binding'])
                  codes_list  = get_expansion(diff['binding']['valueSet'])
                  table_rows.append(f"{obj['title']}|<{diff['binding']['valueSet']}> ({codes_list})")
                  profile_with_category_binding.add(obj['title'])
                except KeyError as e:
                  pass 
                
for profile in my_profiles:
  if profile not in profile_with_category_binding:
    table_rows.append(f"{profile}|-|")

for row in sorted(table_rows):
  print(row)




|Profile|Category Code or Value Set|
|---|---|
US Core ADI DocumentReference Profile|<http://cts.nlm.nih.gov/fhir/ValueSet/2.16.840.1.113762.1.4.1115.25> (None)
US Core AllergyIntolerance Profile|-|
US Core CarePlan Profile|assess-plan|
US Core CareTeam Profile|-|
US Core Coverage Profile|-|
US Core DiagnosticReport Profile for Laboratory Results Reporting|LAB|
US Core DiagnosticReport Profile for Report and Note Exchange|<http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-category> (['LP29684-5(RAD)', 'LP29708-2(Card)', 'LP7839-6(PATH)'])
US Core DocumentReference Profile|<http://hl7.org/fhir/us/core/ValueSet/us-core-documentreference-category> (['clinical-note(Clinical Note)'])
US Core Encounter Profile|-|
US Core Goal Profile|-|
US Core Immunization Profile|-|
US Core Implantable Device Profile|-|
US Core Location Profile|-|
US Core Medication Profile|-|
US Core MedicationDispense Profile|-|
US Core MedicationRequest Profile|<http://hl7.org/fhir/ValueSet/medicationrequest

# Find all element reference types in folder of StructureDefinitions

- for hard coded path search all SD for must support choice elments
- list all reference choices in a md table format in std out

In [ ]:

for i in my_path.glob('Struct*.json'):
    # print(i)
    obj = load(i.open())
    # print(obj['id'])
    #find reference type elements in diff
    for diff in obj['differential']['element']:
        try:
          if diff['mustSupport'] == True:
            # print(i)
            # print(diff['path'])
            #list the choices
            for snapshot in obj['snapshot']['element']:
                if diff['path'] == snapshot['path']:
                   # print(snapshot['path'])
                   for my_type in snapshot['type']:
                        if my_type['code'] == "Reference" and len(my_type['targetProfile']) > 1:
                            print(f"{obj['title']}|{diff['path']}|{', '.join( my_type['targetProfile'])}")
                            break
        except:
            pass

In [71]:
vs={
  "resourceType": "ValueSet",
  "id": "2.16.840.1.113762.1.4.1115.25",
  "meta": {
    "versionId": "8",
    "lastUpdated": "2024-06-29T01:12:54.000-04:00",
    "profile": [ "http://hl7.org/fhir/StructureDefinition/shareablevalueset", "http://hl7.org/fhir/us/cqfmeasures/StructureDefinition/computable-valueset-cqfm", "http://hl7.org/fhir/us/cqfmeasures/StructureDefinition/publishable-valueset-cqfm" ]
  },
  "extension": [ {
    "url": "http://hl7.org/fhir/StructureDefinition/valueset-effectiveDate",
    "valueDate": "2024-06-29"
  } ],
  "url": "http://cts.nlm.nih.gov/fhir/ValueSet/2.16.840.1.113762.1.4.1115.25",
  "identifier": [ {
    "system": "urn:ietf:rfc:3986",
    "value": "urn:oid:2.16.840.1.113762.1.4.1115.25"
  } ],
  "version": "20240629",
  "name": "AdvanceHealthcareDirectiveCategoriesGrouper",
  "title": "Advance Healthcare Directive Categories Grouper",
  "status": "active",
  "date": "2024-06-29T01:12:54-04:00",
  "publisher": "ADVault",
  "expansion": {
    "identifier": "urn:uuid:6503ade0-a9d8-4a9c-b999-8bf417c1da47",
    "timestamp": "2025-02-25T14:33:08-05:00",
    "total": 9,
    "offset": 0,
    "parameter": [ {
      "name": "count",
      "valueInteger": 1000
    }, {
      "name": "offset",
      "valueInteger": 0
    } ],
    "contains": [ {
      "system": "http://loinc.org",
      "version": "2.78",
      "code": "100821-8",
      "display": "National POLST form: portable medical order panel"
    }, {
      "system": "http://loinc.org",
      "version": "2.78",
      "code": "104144-1",
      "display": "Mental health Advance directives"
    }, {
      "system": "http://loinc.org",
      "version": "2.78",
      "code": "42348-3",
      "display": "Advance directives"
    }, {
      "system": "http://loinc.org",
      "version": "2.78",
      "code": "64298-3",
      "display": "Power of attorney"
    }, {
      "system": "http://loinc.org",
      "version": "2.78",
      "code": "81334-5",
      "display": "Patient Personal advance care plan"
    }, {
      "system": "http://loinc.org",
      "version": "2.78",
      "code": "84095-9",
      "display": "Do not resuscitate"
    }, {
      "system": "http://loinc.org",
      "version": "2.78",
      "code": "86533-7",
      "display": "Patient Living will"
    }, {
      "system": "http://loinc.org",
      "version": "2.78",
      "code": "92664-2",
      "display": "Power of attorney and Living will"
    }, {
      "system": "http://loinc.org",
      "version": "2.78",
      "code": "93037-0",
      "display": "Portable medical order form"
    } ]
  }
}

print([f"{contains['code']}({contains['display']})" for contains in vs['expansion']['contains']])



['100821-8(National POLST form: portable medical order panel)', '104144-1(Mental health Advance directives)', '42348-3(Advance directives)', '64298-3(Power of attorney)', '81334-5(Patient Personal advance care plan)', '84095-9(Do not resuscitate)', '86533-7(Patient Living will)', '92664-2(Power of attorney and Living will)', '93037-0(Portable medical order form)']
